In [94]:
#! git clone https://github.com/LeonardoDiCaterina/Neural_Evo_Learn

In [95]:
#! pip install git+https://github.com/DALabNOVA/slim.git

In [96]:
# Third-party imports
import pandas as pd
import torch
from sklearn.model_selection import KFold

# Slim-GSGP imports
from slim_gsgp.datasets.data_loader import load_pandas_df
from slim_gsgp.utils.utils import train_test_split
#from slim_gsgp.main_gp import gp
from slim_gsgp.main_gsgp import gsgp
#from slim_gsgp.main_slim import slim
from slim_gsgp.evaluators.fitness_functions import rmse

import statistics
from collections import defaultdict
import itertools

from sklearn.model_selection import KFold
import numpy as np
import os
import random
from itertools import product

In [100]:
os.chdir(os.path.join(os.getcwd(), os.pardir))
from utils.grid_search import gp_nested_cross_validation
from utils.new_visualization_funcs import *
%cd notebooks/

c:\Users\rafas\Documents\University\NEL\Neural_Evo_Learn\notebooks


c:\Users\rafas\Documents\University\NEL\Neural_Evo_Learn\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning:

This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.



## Load Data

In [99]:
# Reading the desired dataset
df = pd.read_csv("../data/sustavianfeed.csv", sep=';')

# Dropping the first column (index) and renaming the columns
df = df.drop(columns= ['WING TAG', 'EMPTY MUSCULAR STOMACH'])

# Moving crude protein to the end of the dataframe
df = df[[col for col in df.columns if col != 'CRUDE PROTEIN'] + ['CRUDE PROTEIN']]

# Nested CV with Grid Search

In [16]:
seed = 42
random.seed(seed)

# Edit the name and log directory based on the model you want to run

#MODEL_NAME = 'GP'
MODEL_NAME = 'GSGP'
#MODEL_NAME = 'SLIM-GSGP'

DATASET_NAME = MODEL_NAME +'_sustavianfeed'
LOG_DIR = './log/' + MODEL_NAME + '/'

LOG_LEVEL = 2
if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)

In [8]:
k_outer = 10
k_inner = 5

In [9]:
# Turning df into X and y torch.Tensors
X, y = load_pandas_df(df, X_y=True)

In [10]:
FITNESS_FUNCTION = 'rmse'
MINIMIZATION = True

total_instances = X.shape[0]
outer_test_size = total_instances // k_outer
outer_train_size = total_instances - outer_test_size
inner_val_size = outer_train_size // k_inner
inner_train_size = outer_train_size - inner_val_size

print(f'Total Instances:\t{total_instances}\n--')
print(f'Outer Train set:\t{outer_train_size}')
print(f'Test set:\t\t{outer_test_size}\n--')
print(f'Inner Train set:\t{inner_train_size}')
print(f'Validation set:\t\t{inner_val_size}\n')

Total Instances:	96
--
Outer Train set:	87
Test set:		9
--
Inner Train set:	70
Validation set:		17



In [11]:
POP_SIZE = 20

fixed_params = {
    # ---
    # Search Space
    'initializer': 'rhh',
    'init_depth': 2,
    'tree_constants': [random.uniform(-1, 1) for _ in range(9)],
    'tree_functions': ['add', 'subtract','multiply','divide'],
    #'prob_const': 0.1,
    # ---
    # Problem Instance
    'dataset_name': DATASET_NAME,
    'fitness_function': 'rmse',
    'minimization': True,
    # ---
    # Model instance
    #'tournament_size': int(POP_SIZE*0.02) if POP_SIZE>100 else 2,
    'pop_size': POP_SIZE,


    # ---
    # Solve settings
    'n_iter': 500,
    'elitism': True,
    'n_elites': 2,
    'test_elite': True,
    'log_level': LOG_LEVEL,
    'verbose': 0,
    'n_jobs': 1,

    # ---
    # GP unique settings
    #'max_depth': 10,
    #'p_xo' : 0.5,


    # ---
    # GSGP unique settings
    #'p_xo' : 0.5,
    'ms_lower': 0,
    'ms_upper': 0.5,
    #'reconstruct': False,


    # ---
    # SLIM unique settings
    #'max_depth': 10,
    #'#p_inflate': 0.70,
    #'slim_version': 'SLIM+SIG2',
    #'copy_parent': True,
    # 'ms_lower': 0,
    # 'ms_upper': 1,
    #'reconstruct': False,


}


param_grid = {
        'p_xo' : [0.5, 0.7],
        'tournament_size': [int(POP_SIZE*0.20), int(POP_SIZE*0.10), int(POP_SIZE*0.15)],
        'prob_const': [0.1, 0.7],

}


In [ ]:
outer_results = gp_nested_cross_validation(X, y, gp_model=gsgp, k_outer=k_outer, k_inner=k_inner, fixed_params=fixed_params, param_grid=param_grid, seed=seed, LOG_DIR=LOG_DIR, DATASET_NAME=DATASET_NAME)

Saving results and configs to a .csv 

In [ ]:
outer_results_df = pd.DataFrame(outer_results)
outer_results_df.to_csv(LOG_DIR+DATASET_NAME+'_outer_results.csv', index=False)


## Visualizations 

In [ ]:
# 0  - Algorithm
# 1  - Instance ID
# 2  - Dataset
# 3  - Seed
# 4  - Generation
# 5  - Fitness
# 6  - Running time
# 7  - Population nodes
# 8  - Test fitness
# 9  - Elite nodes
# 10 - niche entropy
# 11 - sd(pop.fit)
# 12 - Log level 

c:\Users\rafas\Documents\University\NEL\Neural_Evo_Learn\notebooks


c:\Users\rafas\Documents\University\NEL\Neural_Evo_Learn\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [22]:
train_test_best_combs(model_name=MODEL_NAME)         

In [25]:
def niche_entropy(k_outer, model_name, rows=5, cols=2):
    LOG_DIR = './log/' + model_name + '/' + model_name +'_sustavianfeed'

    assert rows*cols==k_outer, "The number of combinations does not correspond to the grid size defined (rows/cols)."

    fig = sp.make_subplots(rows=rows, cols=cols, 
                           #subplot_titles=[f"Combination index: {unique_setting_df[unique_setting_df[0]==comb].index[0]}" 
                           #                for comb in dif_combs]
                            )

    for i_outer in range(k_outer):
        LOG_PATH = LOG_DIR + f'_outer_{i_outer}.csv'
        df = pd.read_csv(LOG_PATH, header=None)
        y = df
        row = (i_outer // cols) + 1
        col = (i_outer % cols) + 1

        y = y.iloc[:, 10].values,
        y_float = (
            pd.Series(y)
            .str.replace(r'^tensor\(|\)$', '', regex=True)  # drop wrapper
            .astype(float)
            .to_numpy()
        )

        
        fig.add_trace(
            go.Scatter( 
                y=y_float,
                mode='lines',
                name='Niche Entropy',
                line=dict(color='orange'),
                showlegend=(i_outer == 0)), row=row, col=col
                )
    
        fig.update_layout(
            height=150 * rows,
            width=250 * cols,
            margin=dict(t=50),
            title_text=f'{model_name} - Niche Entropy (x=Generation, y=Entropy)',
        )
    
    fig.show()

In [26]:
niche_entropy(k_outer=10, model_name='GSGP')